## Integrating Curve StableSwap - METAPOOL Factory

In [1]:
import requests
import numpy as np
import pandas as pd
from web3 import Web3
from tqdm import tqdm
from tqdm import tqdm 
import os

w3 = Web3()

In [2]:
global all_events
all_events = []

def get_ethers_ABI_and_hash(event_abi_json):
    names = []
    types = []
    indexeds = []

    for input in event_abi_json['inputs']: 
        names.append(input['name'])
        types.append(input['type'])
        if input['indexed']:
            indexeds.append('indexed')
        else:
            indexeds.append('')
    
    inupt_types = []
    for i in range(len(names)):
        tstr = (f'{types[i]} {indexeds[i]} {names[i]}')
        inupt_types.append(' '.join(tstr.split()))
    
    event_signature_hash = Web3.keccak(text = f"{event_abi_json['name']}({','.join(types)})").hex()
    ethersJS_abi = f"{event_abi_json['type']} {event_abi_json['name']}({', '.join(inupt_types)})"
    
    return [ethersJS_abi, event_signature_hash]


def decode_raw_log(log, events, event_name):

    indexed_type = []
    indexed_name = []
    data_type = []
    data_name = []

    for input in events[event_name]['jsonABI']['inputs']:
        if input['indexed']:
            indexed_type.append(input['type'])
            indexed_name.append(input['name']) 
        else:
            data_type.append(input['type'])
            data_name.append(input['name']) 
                
    # topics
    if not len(indexed_type) == len(log['TOPICS']) - 1:
        raise(TypeError, "The ABI and data do not match")
    
    decoded_topics = []
    for i in range(len(indexed_type)):
        decoded_topics.append(w3.codec.decode([indexed_type[i]], w3.to_bytes(hexstr = log['TOPICS'][i+1]))[0])
    
    decoded_data = w3.codec.decode(data_type, w3.to_bytes(hexstr = log['DATA']))

    decoded_log = {'event' : event_name, 'logIndex' : log['INDEX'], 'address' : log['ADDRESS']}
    for i in range(len(indexed_name)):
        decoded_log.update({indexed_name[i] : decoded_topics[i]})

    for i in range(len(data_name)):
        decoded_log.update({data_name[i] : decoded_data[i]})

    return decoded_log 


def txt_to_str(path):
    txt = open(path, 'r')
    data = txt.read()
    return data

def get_account_key(account_name: str) -> str:
    with open(
        os.path.expanduser(f"~/.cryptocompare/{account_name}-key"),
        encoding="utf-8",
    ) as file:
        key = file.read().rstrip("\n")
    return key


def get_events(factory_name, FACTORY_ABI): 
    # Connect to Web3
    factorycontract = w3.eth.contract(abi= FACTORY_ABI) 

    # Get All Factory Events
    events = {}
    for event in factorycontract.events._events:
        sub_abi, event_hash = get_ethers_ABI_and_hash(event)
        print(sub_abi, event_hash)
        if any( i in sub_abi for i in ['AddLiquidity', 'BasePoolAdded', 'PlainPoolDeployed', 'MetaPoolDeployed',
                       'ApplyNewFee', 'NewFee', 'PoolAdded', 'RemoveLiquidity', 'RemoveLiquidityOne',
                       'RemoveLiquidityImbalance', 'TokenExchange', 'TokenExchangeUnderlying', 'Transfer']):
            all_events.append([factory_name, sub_abi, event_hash])
        
        events.update({event['name'] : {'event_hash' : event_hash,
                    'ethersABI' : sub_abi,
                    'jsonABI' : event}})
    return events

In [3]:
def get_CC_block(block_number, api_key = get_account_key('hmahmoud')):
    response = requests.get(f'https://data-api.cryptocompare.com/onchain/v1/block/2?block_number={block_number}&groups=ID,METADATA,TRANSACTIONS,ORPHAN_TRACES,UNCLES,WITHDRAWALS&api_key={api_key}')
    if response.ok:
        block = response.json()['Data']
        return block
    else: print(requests.get(f'https://data-api.cryptocompare.com/onchain/v1/block/2?block_number={block_number}&groups=ID,METADATA,TRANSACTIONS,ORPHAN_TRACES,UNCLES,WITHDRAWALS&api_key={api_key}'))

# CONNECT TO WEB3 - NEW TO THIS NOTEBOOK
rpc_url = ('https://frequent-icy-spree.discover.quiknode.pro/6f6aa80d72ff259ce08ef10c08cfa9d7f07b3dda/')
web3 = Web3(Web3.HTTPProvider(rpc_url))

### Getting Factory Events and All Markets

In [4]:
print("---- Meta Pool Events -------")
META_1_FACTORY_ADDRESS = '0x0959158b6040D32d04c301A72CBFD6b39E21c9AE' #CRUVE METAPOOL FACTORY 
META_1_FACTORY_ABI = txt_to_str('./abis/metapool_v1_factory_abi.txt')
META_1_MARKET_ABI = txt_to_str('./abis/metapool_v1_market_abi.txt')
META_1_events = get_events('meta', META_1_FACTORY_ABI)
META_1_events1 = get_events('meta', META_1_MARKET_ABI )

---- Meta Pool Events -------
event BasePoolAdded(address base_pool, address implementat) 0xd1951e19788b24c3c75b3c6c19a795cd312b1523d09076fc41e432cd3ccdc14c
event MetaPoolDeployed(address coin, address base_pool, uint256 A, uint256 fee, address deployer) 0x01f31cd2abdeb4e5e10ba500f2db0f937d9e8c735ab04681925441b4ea37eda5
event Transfer(address indexed sender, address indexed receiver, uint256 value) 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef
event Approval(address indexed owner, address indexed spender, uint256 value) 0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925
event TokenExchange(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bought) 0x8b3e96f2b889fa771c53c981b40daf005f63f637f1869f707052d15a3dd97140
event TokenExchangeUnderlying(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bought) 0xd013ca23e77a65003c2c659c5442c00c805371b7fc1ebd4c206c41d1536bd90b


In [5]:
print("---- Meta Pool Events -------")
META_FACTORY_ADDRESS = '0xB9fC157394Af804a3578134A6585C0dc9cc990d4' #CRUVE METAPOOL FACTORY 
META_FACTORY_ABI = txt_to_str('./abis/metapool_factory_abi.txt')
META_MARKET_ABI = txt_to_str('./abis/metapool_markets_abi.txt')
META_events = get_events('meta', META_FACTORY_ABI)
META_events1 = get_events('meta', META_MARKET_ABI )

---- Meta Pool Events -------
event BasePoolAdded(address base_pool) 0xcc6afdfec79da6be08142ecee25cf14b665961e25d30d8eba45959be9547635f
event PlainPoolDeployed(address[4] coins, uint256 A, uint256 fee, address deployer) 0x5b4a28c940282b5bf183df6a046b8119cf6edeb62859f75e835eb7ba834cce8d
event MetaPoolDeployed(address coin, address base_pool, uint256 A, uint256 fee, address deployer) 0x01f31cd2abdeb4e5e10ba500f2db0f937d9e8c735ab04681925441b4ea37eda5
event LiquidityGaugeDeployed(address pool, address gauge) 0x656bb34c20491970a8c163f3bd62ead82022b379c3924960ec60f6dbfc5aab3b
event Transfer(address indexed sender, address indexed receiver, uint256 value) 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef
event Approval(address indexed owner, address indexed spender, uint256 value) 0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925
event TokenExchange(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bought) 0x8b3e96f2

In [6]:
print("---- Registry Pool Events -------")
REG_FACTORY_ADDRESS = '0x90E00ACe148ca3b23Ac1bC8C240C2a7Dd9c2d7f5'
REG_FACTORY_ABI = txt_to_str('./abis/registry_factory_abi.txt')
REG_MARKET_ABI = txt_to_str('./abis/registry_market_abi.txt')
REG_events = get_events('registry', REG_FACTORY_ABI)
REG_events1 = get_events('registry', REG_MARKET_ABI)

---- Registry Pool Events -------
event PoolAdded(address indexed pool, bytes rate_method_id) 0xe485c16479ab7092c0b3fc4649843c06be7f072194675261590c84473ab0aea9
event PoolRemoved(address indexed pool) 0x4106dfdaa577573db51c0ca93f766dbedfa0758faa2e7f5bcdb7c142be803c3f
event TokenExchange(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bought) 0x8b3e96f2b889fa771c53c981b40daf005f63f637f1869f707052d15a3dd97140
event AddLiquidity(address indexed provider, uint256[3] token_amounts, uint256[3] fees, uint256 invariant, uint256 token_supply) 0x423f6495a08fc652425cf4ed0d1f9e37e571d9b9529b1c1c23cce780b2e7df0d
event RemoveLiquidity(address indexed provider, uint256[3] token_amounts, uint256[3] fees, uint256 token_supply) 0xa49d4cf02656aebf8c771f5a8585638a2a15ee6c97cf7205d4208ed7c1df252d
event RemoveLiquidityOne(address indexed provider, uint256 token_amount, uint256 coin_amount) 0x9e96dd3b997a2a257eec4df9bb6eaf626e206df5f543bd963682d143300be310
event R

In [7]:
print("------- Registery Pool Events (2) ----------")
REG_FACTORY_ADDRESS = '0x90E00ACe148ca3b23Ac1bC8C240C2a7Dd9c2d7f5'
REG_FACTORY_ABI = txt_to_str('./abis/registry_factory_abi.txt')
REG_MARKET_ABI = txt_to_str('./abis/registry_market_abi_2.txt')
REG_events = get_events('registry', REG_FACTORY_ABI)
REG_events11 = get_events('registry', REG_MARKET_ABI)

------- Registery Pool Events (2) ----------
event PoolAdded(address indexed pool, bytes rate_method_id) 0xe485c16479ab7092c0b3fc4649843c06be7f072194675261590c84473ab0aea9
event PoolRemoved(address indexed pool) 0x4106dfdaa577573db51c0ca93f766dbedfa0758faa2e7f5bcdb7c142be803c3f
event TokenExchange(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bought) 0x8b3e96f2b889fa771c53c981b40daf005f63f637f1869f707052d15a3dd97140
event AddLiquidity(address indexed provider, uint256[2] token_amounts, uint256[2] fees, uint256 invariant, uint256 token_supply) 0x26f55a85081d24974e85c6c00045d0f0453991e95873f52bff0d21af4079a768
event RemoveLiquidity(address indexed provider, uint256[2] token_amounts, uint256[2] fees, uint256 token_supply) 0x7c363854ccf79623411f8995b362bce5eddff18c927edc6f5dbbb5e05819a82c
event RemoveLiquidityOne(address indexed provider, uint256 token_amount, uint256 coin_amount, uint256 token_supply) 0x5ad056f2e28a8cec232015406b843668c1e36cd

In [8]:
print("---- Stableswap Pool Events -------")
STABLESWAP_FACTORY_ADDRESS = '0x6A8cbed756804B16E05E741eDaBd5cB544AE21bf'
STABLESWAP_FACTORY_ABI = txt_to_str('./abis/stableswap-ng_factory_abi.txt')
STABLESWAP_MARKET_ABI = txt_to_str('./abis/stableswap_ng_market_abi.txt')
STABLESWAP_events = get_events('stable', STABLESWAP_FACTORY_ABI)
STABLESWAP_events1 = get_events('stable', STABLESWAP_MARKET_ABI)

---- Stableswap Pool Events -------
event BasePoolAdded(address base_pool) 0xcc6afdfec79da6be08142ecee25cf14b665961e25d30d8eba45959be9547635f
event PlainPoolDeployed(address[] coins, uint256 A, uint256 fee, address deployer) 0xd1d60d4611e4091bb2e5f699eeb79136c21ac2305ad609f3de569afc3471eecc
event MetaPoolDeployed(address coin, address base_pool, uint256 A, uint256 fee, address deployer) 0x01f31cd2abdeb4e5e10ba500f2db0f937d9e8c735ab04681925441b4ea37eda5
event LiquidityGaugeDeployed(address pool, address gauge) 0x656bb34c20491970a8c163f3bd62ead82022b379c3924960ec60f6dbfc5aab3b
event Transfer(address indexed sender, address indexed receiver, uint256 value) 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef
event Approval(address indexed owner, address indexed spender, uint256 value) 0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925
event TokenExchange(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bought) 0x8b3

In [9]:
print("---- CrvUSD Pool Events -------")
CRVUSD_FACTORY_ADDRESS = '0x4F8846Ae9380B90d2E71D5e3D042dff3E7ebb40d' 
CRVUSD_FACTORY_ABI = txt_to_str('./abis/crvusd_factory_abi.txt')
CRVUSD_MARKET_ABI = txt_to_str('./abis/crvusd_market_abi.txt')
CRVUSD_events = get_events('crvusd', CRVUSD_FACTORY_ABI)
CRVUSD_events1 = get_events('crvusd', CRVUSD_MARKET_ABI)

---- CrvUSD Pool Events -------
event BasePoolAdded(address base_pool) 0xcc6afdfec79da6be08142ecee25cf14b665961e25d30d8eba45959be9547635f
event PlainPoolDeployed(address[4] coins, uint256 A, uint256 fee, address deployer, address pool) 0xb8f6972d6e56d21c47621efd7f02fe68f07a17c999c42245b3abd300f34d61eb
event MetaPoolDeployed(address coin, address base_pool, uint256 A, uint256 fee, address deployer) 0x01f31cd2abdeb4e5e10ba500f2db0f937d9e8c735ab04681925441b4ea37eda5
event LiquidityGaugeDeployed(address pool, address gauge) 0x656bb34c20491970a8c163f3bd62ead82022b379c3924960ec60f6dbfc5aab3b
event Transfer(address indexed sender, address indexed receiver, uint256 value) 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef
event Approval(address indexed owner, address indexed spender, uint256 value) 0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925
event TokenExchange(address indexed buyer, int128 sold_id, uint256 tokens_sold, int128 bought_id, uint256 tokens_bo

In [52]:
pd.DataFrame(all_events, columns=['factory', 'event', 'topic'])

,factory,event,topic
0,meta,"event BasePoolAdded(address base_pool, address...",0xd1951e19788b24c3c75b3c6c19a795cd312b1523d090...
1,meta,"event MetaPoolDeployed(address coin, address b...",0x01f31cd2abdeb4e5e10ba500f2db0f937d9e8c735ab0...
2,meta,"event Transfer(address indexed sender, address...",0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
3,meta,"event TokenExchange(address indexed buyer, int...",0x8b3e96f2b889fa771c53c981b40daf005f63f637f186...
4,meta,event TokenExchangeUnderlying(address indexed ...,0xd013ca23e77a65003c2c659c5442c00c805371b7fc1e...
...,...,...,...
97,crvusd,event RemoveLiquidity(address indexed provider...,0x7c363854ccf79623411f8995b362bce5eddff18c927e...
98,crvusd,event RemoveLiquidityOne(address indexed provi...,0x5ad056f2e28a8cec232015406b843668c1e36cda5981...
99,crvusd,event RemoveLiquidityImbalance(address indexed...,0x2b5508378d7e19e0d5fa338419034731416c4f5b219a...
100,crvusd,event CommitNewFee(uint256 new_fee),0x878eb36b3f197f05821c06953d9bc8f14b332a227b1e...


## GET MARKET REDIS

In [10]:
def discover_new_instruments(pool, FACTORY_ADDRESS, block, transaction, exchange_name, market, market0, markets_redis):

    if pool == 'PoolAdded':
        instrumentId = market['pool']
        
        market_contract = web3.eth.contract(address = Web3.to_checksum_address(instrumentId), abi= REG_MARKET_ABI)
        
        A = market_contract.functions.A().call()
        fee = market_contract.functions.fee().call()
        
        tokens = []
        for i in range(4):
            # MAKE SURE THIS IS WORKING OR OTHERWISE WE WON'T HAVE TOKENS
            try:
                pool_token = market_contract.functions.coins(i).call()
                tokens.append(pool_token)
            except:
                pass

        deployer = ''
        base_pool_if_meta = None
        pool_type = 'Registry_Pool'

    if pool == 'PlainPoolDeployed':
        if FACTORY_ADDRESS == CRVUSD_FACTORY_ADDRESS.lower():  #CRV USD HAS DIFFERENT ARRANGEMENT
            instrumentId = market['pool']
        else:
            instrumentId = market0['address'].lower()

        tokens = list([i.lower() for i in market['coins']])
        fee =  market['fee']
        A =  market['A']
        deployer = market['deployer']
        base_pool_if_meta = None
        pool_type = 'PlainPool'
    
    if pool == 'MetaPoolDeployed': 
        instrumentId = market0['address'].lower()
        fee =  market['fee']
        A =  market['A']
        deployer = market['deployer']
        base_pool_if_meta = market['base_pool']

        base_pool = [pool for pool in markets_redis if pool['instrumentId'] == market['base_pool']][0]
        tokens = [market['coin'].lower()] + base_pool['tokens']
        pool_type = 'MetaPool'

    while '0x0000000000000000000000000000000000000000' in tokens:
        tokens.remove('0x0000000000000000000000000000000000000000')
    
    # DELETE REGISTERY POOL IF POOL IS METAPOOL V1 
    if len(markets_redis) > 0: 
        for pool in markets_redis:
            if pool['instrumentId'] == instrumentId:
                print(pool)
                if pool['factoryAddress'] == REG_FACTORY_ADDRESS.lower():
                    markets_redis.remove(pool)
                else: 
                    raise KeyError('Dublicated Pools')

    if pool_type == 'Registry_Pool':
        lp_token = ''
    
    else : lp_token = instrumentId

    new_market = {'exchangeInternalName' : exchange_name,
                'exchangeOwnedBy' : 'Curve',
                'factoryAddress' : FACTORY_ADDRESS,
                'executedTS' : block['TIMESTAMP'], 
                'executedNS' : 0,
                'transactionHash' : transaction['HASH'], 
                'blockNumber' : block['NUMBER'], 
                'from' : transaction['FROM_ADDRESS'], 
                'instrumentId' :  instrumentId,
                'tokens': tokens,
                'fee': fee,
                'A' : A,
                'lp_token_address': lp_token,
                'base_pool_if_meta': base_pool_if_meta,                                
                'base_pool_functionality': [],
                'poolType' : pool_type,
                'deployer' : deployer,
                'receivedMS' : block['RECEIVED_TIMESTAMP'], 
                'fullBlockOnDataApi.ASSET_ID' : block['ASSET_ID'], 
                'fullBlockOnDataApi.PROVIDER_KEY' : block['PROVIDER_KEY'],
                    }

    return new_market


def create_combinations_of_markets(new_market, markets_redis, block):

    coins = new_market['tokens']
    InstrumentId = new_market['instrumentId']

    if len(coins) > 2:  
        for i in range(len(coins)):
            for j in range(i + 1, len(coins)):
                market_copy = new_market.copy()
                market_copy['instrumentId'] = f"{InstrumentId}_{str(block['ASSET_ID'])}_{coins[i]}_{coins[j]}"
                market_copy['tokens'] = [coins[i], coins[j]]
                markets_redis.append(market_copy)
    else:
        pass

    
def add_base_pool_functionality(market, markets_redis, transaction, block, factory_address):
    for pool in markets_redis:
        if pool['instrumentId'] == market['base_pool']:
            pool['base_pool_functionality'].append({
                    'factory_Address': factory_address,
                    'executedTS' : block['TIMESTAMP'], 
                    'transactionHash': transaction['HASH']
                     })


In [10]:
# Begining of Registery

BLOCKS_TO_START_WITH = [
12195812, 12195888, 12195935, 12196005, 12196249, 12196340, 12196401, 12196525, 12196644, 12196745, 12196969, 	
12197228, 12197351, 12197486, 12197579, 12197620, 12197653, 12197697, 12197727, 12197761, 12197790, 12197825,
12197858, 12197887, 12197939, 12197964, 12198010, 12198042, 12198091, 12198122, 12198164, 12198209, 12201258,
12336658, 12336733, 12523127, 12523155, 12661180, 12844479, 12923457, 13139859, 13393350, 13644715, 13644772,
13933720, 14700596, 14700602, 14967642, 15749716, 16100183, 16832765, 17139535]

START_BLOCK = 10809473


In [11]:
markets_redis = []

for block_number in tqdm([
    14700602, 14967642, 15455801
    ]): 

    block = get_CC_block(block_number)
    for transaction in block['TRANSACTIONS']:
        log_num = 0 
        for log in transaction['LOGS']:

            if log['TOPICS'] == [META_events1['Transfer']['event_hash']]:
                log_num = log_num + 1

            # ------------------------------
            # REGISTRY ADDRESS INTEGRATION
            # ------------------------------
            if log['ADDRESS'] in REG_FACTORY_ADDRESS.lower():
                # IF EVENT IS PoolAdded
                if REG_events['PoolAdded']['event_hash'] in log['TOPICS']:
                    market = decode_raw_log([log for log in transaction['LOGS']if REG_events['PoolAdded']['event_hash'] in log['TOPICS']][0], REG_events, 'PoolAdded')
                
                    new_market = discover_new_instruments('PoolAdded', REG_FACTORY_ADDRESS.lower(), block, transaction, 'Curve_Registry', market, None, markets_redis)
                    markets_redis.append(new_market)
                    create_combinations_of_markets(new_market, markets_redis, block)
            
                # IF EVENT IS PoolRemoved
                if REG_events['PoolRemoved']['event_hash'] in log['TOPICS']:
                    pass

            # ------------------------------
            # META__STABLESWAP__CRVUSD_FACTORY_ADDRESS_INTEGRATION
            # ------------------------------
            if (log['ADDRESS'] in META_FACTORY_ADDRESS.lower() or
                log['ADDRESS'] in STABLESWAP_FACTORY_ADDRESS.lower() or
                log['ADDRESS'] in CRVUSD_FACTORY_ADDRESS.lower() or
                log['ADDRESS'] in META_1_FACTORY_ADDRESS.lower()): #NEW FACTORY FOUND

                if log['ADDRESS'] == META_1_FACTORY_ADDRESS.lower(): 
                    exchange_name = 'Curve_MetaPoolFactory'
                    factory_address = META_1_FACTORY_ADDRESS.lower()
                
                elif log['ADDRESS'] == META_FACTORY_ADDRESS.lower(): 
                    exchange_name = 'Curve_MetaPoolFactory'
                    factory_address = META_FACTORY_ADDRESS.lower()

                elif log['ADDRESS'] == CRVUSD_FACTORY_ADDRESS.lower(): 
                    exchange_name = 'Curve_crvUSD_Factory'
                    factory_address = CRVUSD_FACTORY_ADDRESS.lower() 
                
                elif log['ADDRESS'] == STABLESWAP_FACTORY_ADDRESS.lower(): 
                    exchange_name = 'Curve_StableSwap-NG'
                    factory_address = STABLESWAP_FACTORY_ADDRESS.lower()
                
                else: 
                    raise KeyError('Unrecognised Factory')

                # IF EVENT IS PlainPoolDeployed (META)
                if log['TOPICS'] == [META_events['PlainPoolDeployed']['event_hash']]:

                    market0 = decode_raw_log([log for log in transaction['LOGS']if META_events1['Transfer']['event_hash'] in log['TOPICS']][log_num-1], META_events1, 'Transfer')
                    market = decode_raw_log(log, META_events, 'PlainPoolDeployed')
                    
                    new_market = discover_new_instruments('PlainPoolDeployed', factory_address, block, transaction, exchange_name, market, market0, markets_redis)
                    markets_redis.append(new_market)
                    create_combinations_of_markets(new_market, markets_redis, block)

                # IF EVENT IS PlainPoolDeployed (STABLESWAP-NG)
                if log['TOPICS'] == [STABLESWAP_events['PlainPoolDeployed']['event_hash']]:

                    market0 = decode_raw_log([log for log in transaction['LOGS']if META_events1['Transfer']['event_hash'] in log['TOPICS']][log_num-1], STABLESWAP_events1, 'Transfer')
                    market = decode_raw_log(log, STABLESWAP_events, 'PlainPoolDeployed')
                    
                    new_market = discover_new_instruments('PlainPoolDeployed', factory_address, block, transaction, exchange_name, market, market0, markets_redis)
                    markets_redis.append(new_market)
                    create_combinations_of_markets(new_market, markets_redis, block)
            
                # IF EVENT IS PlainPoolDeployed (CRVUSD)
                if log['TOPICS'] == [CRVUSD_events['PlainPoolDeployed']['event_hash']]:

                    market = decode_raw_log(log, CRVUSD_events, 'PlainPoolDeployed')
                
                    new_market = discover_new_instruments('PlainPoolDeployed', factory_address, block, transaction, exchange_name, market, market, markets_redis)
                    markets_redis.append(new_market)
                    create_combinations_of_markets(new_market, markets_redis, block)
                
                # IF EVENT IS MetaPoolDeployed (ALL META POOLS ARE SIMILLAR)
                if log['TOPICS'] == [META_events['MetaPoolDeployed']['event_hash']]:

                    market0 = decode_raw_log([log for log in transaction['LOGS']if META_events1['Transfer']['event_hash'] in log['TOPICS']][log_num-1], META_events1, 'Transfer')
                    market = decode_raw_log(log, META_events, 'MetaPoolDeployed')

                    new_market =   discover_new_instruments("MetaPoolDeployed", factory_address, block, transaction, exchange_name, market,  market0, markets_redis)
                    markets_redis.append(new_market)
                    create_combinations_of_markets(new_market, markets_redis, block)

                # IF EVENT IS BasePoolAdded (ALL BASE POOLS ARE SIMILLAR)
                if log['TOPICS'] == [META_events['BasePoolAdded']['event_hash']]:

                    market = decode_raw_log(log, META_events, 'BasePoolAdded')
                    add_base_pool_functionality(market, markets_redis, transaction, block, factory_address)

                if log['TOPICS'] == [META_1_events['BasePoolAdded']['event_hash']]:
                    
                    market = decode_raw_log(log, META_1_events, 'BasePoolAdded')
                    add_base_pool_functionality(market, markets_redis, transaction, block, factory_address)

             # LP TOKENS INTEGRATION
            if log['TOPICS'][0] == REG_events1['AddLiquidity']['event_hash']: 
                add_liquidty = decode_raw_log(log, REG_events1, 'AddLiquidity')
                for log in transaction['LOGS']:
                    if log['TOPICS'][0] == CRVUSD_events1['Transfer']['event_hash']:
                        log_2 = decode_raw_log(log, CRVUSD_events1, 'Transfer')
                        if (log_2['sender'] == '0x0000000000000000000000000000000000000000') and (log_2['receiver'] == add_liquidty['provider']):
                            lp_token = log_2["address"]
                
                for market in markets_redis:
                    if market['instrumentId'] == add_liquidty['address']:
                        market['lpToken'] = lp_token

100%|██████████| 3/3 [00:07<00:00,  2.58s/it]


In [14]:
pd.DataFrame(markets_redis)

,exchangeInternalName,exchangeOwnedBy,factoryAddress,executedTS,executedNS,transactionHash,blockNumber,from,instrumentId,tokens,fee,A,lp_token_address,base_pool_if_meta,base_pool_functionality,poolType,deployer,receivedMS,fullBlockOnDataApi.ASSET_ID,fullBlockOnDataApi.PROVIDER_KEY
0,Curve_Registry,Curve,0x90e00ace148ca3b23ac1bc8c240c2a7dd9c2d7f5,1651526157,0,0x56e6151b459e8a8532f56006a3c09b581a4cdf99f729...,14700602,0x7eeac6cddbd1d0b8af061742d41877d7f707289a,0x1005f7406f32a61bd760cfa14accd2737913d546,"[0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, 0...",3000000,1500,,None,[],Registry_Pool,,1692701334,2,quicknode
1,Curve_Registry,Curve,0x90e00ace148ca3b23ac1bc8c240c2a7dd9c2d7f5,1655298483,0,0xfb48b7ab14629aece9b6692497e5a1555548088d42ee...,14967642,0xbabe61887f1de2713c6f97e567623453d3c79f67,0xdcef968d416a41cdac0ed8702fac8128a64241a2,"[0x853d955aCEf822Db058eb8505911ED77F175b99e, 0...",1000000,1500,,None,[],Registry_Pool,,1692701334,2,quicknode
2,Curve_MetaPoolFactory,Curve,0xb9fc157394af804a3578134a6585c0dc9cc990d4,1662075789,0,0x2a65604a7112c48d17987941a0187447af3b2bbc65cc...,15455801,0x962228a90eac69238c7d1f216d80037e61ea9255,0xe57180685e3348589e9521aa53af0bcd497e884d,"[0x865377367054516e17014ccded1e7d814edc9ce4, 0...",4000000,200,0xe57180685e3348589e9521aa53af0bcd497e884d,0xdcef968d416a41cdac0ed8702fac8128a64241a2,[],MetaPool,0x962228a90eac69238c7d1f216d80037e61ea9255,1692701334,2,quicknode
3,Curve_MetaPoolFactory,Curve,0xb9fc157394af804a3578134a6585c0dc9cc990d4,1662075789,0,0x2a65604a7112c48d17987941a0187447af3b2bbc65cc...,15455801,0x962228a90eac69238c7d1f216d80037e61ea9255,0xe57180685e3348589e9521aa53af0bcd497e884d_2_0...,"[0x865377367054516e17014ccded1e7d814edc9ce4, 0...",4000000,200,0xe57180685e3348589e9521aa53af0bcd497e884d,0xdcef968d416a41cdac0ed8702fac8128a64241a2,[],MetaPool,0x962228a90eac69238c7d1f216d80037e61ea9255,1692701334,2,quicknode
4,Curve_MetaPoolFactory,Curve,0xb9fc157394af804a3578134a6585c0dc9cc990d4,1662075789,0,0x2a65604a7112c48d17987941a0187447af3b2bbc65cc...,15455801,0x962228a90eac69238c7d1f216d80037e61ea9255,0xe57180685e3348589e9521aa53af0bcd497e884d_2_0...,"[0x865377367054516e17014ccded1e7d814edc9ce4, 0...",4000000,200,0xe57180685e3348589e9521aa53af0bcd497e884d,0xdcef968d416a41cdac0ed8702fac8128a64241a2,[],MetaPool,0x962228a90eac69238c7d1f216d80037e61ea9255,1692701334,2,quicknode
5,Curve_MetaPoolFactory,Curve,0xb9fc157394af804a3578134a6585c0dc9cc990d4,1662075789,0,0x2a65604a7112c48d17987941a0187447af3b2bbc65cc...,15455801,0x962228a90eac69238c7d1f216d80037e61ea9255,0xe57180685e3348589e9521aa53af0bcd497e884d_2_0...,"[0x853d955aCEf822Db058eb8505911ED77F175b99e, 0...",4000000,200,0xe57180685e3348589e9521aa53af0bcd497e884d,0xdcef968d416a41cdac0ed8702fac8128a64241a2,[],MetaPool,0x962228a90eac69238c7d1f216d80037e61ea9255,1692701334,2,quicknode


In [12]:
def get_side_price_quantity_quote_fee(tokens_bought, tokens_bought_id, tokens_sold, tokens_sold_id):
    if tokens_bought_id < tokens_sold_id: 
        side = 'BUY'
        price = tokens_sold / tokens_bought
        quantity = tokens_bought
        quote_quantity = tokens_sold
    elif tokens_bought_id > tokens_sold_id: 
        side = 'SELL'
        price = tokens_bought / tokens_sold
        quantity = tokens_sold
        quote_quantity = tokens_bought
    else: 
        side = 'UNKNOWN'
    
    return side, price, quantity, quote_quantity


def convert_external_swap_to_internal(log, events, event_name, transaction, block, market_redis, exchange_internal_name):
    swap = decode_raw_log(log, events, event_name)

    if swap['address'] in list(pd.DataFrame(market_redis)['instrumentId']):
        market = [i for i in market_redis if i['instrumentId'] == swap['address']][0]
    
        tokens_bought = swap['tokens_bought']
        tokens_bought_id = swap['bought_id']
        tokens_sold = swap['tokens_sold']
        tokens_sold_id = swap['sold_id']

        market_fee_pectentage = market['fee'] * 10**-8 
        market_fee_value = format(market_fee_pectentage * tokens_bought, '.0f')

        side, price, quantity, quote_quantity = get_side_price_quantity_quote_fee(tokens_bought, tokens_bought_id, tokens_sold, tokens_sold_id)
        
        token0_i, token1_i = sorted([tokens_bought_id, tokens_sold_id])
        token_address0 = market['tokens'][token0_i]
        token_address1 = market['tokens'][token1_i]

        internal_trade_object = {'exchangeInternalName' : exchange_internal_name,
                    'fullBlockOnDataApi.ASSET_ID': block['ASSET_ID'], 
                    'instrumentID' : f"{swap['address']}_{block['ASSET_ID']}_{token_address0}_{token_address1}", 
                    'side' : side, 
                    'tradeID' : f"{transaction['HASH']}-{log['INDEX']}", 
                    'executedTS' : block['TIMESTAMP'], 
                    'executedNS' : 0,
                    'transactionHash' : transaction['HASH'], 
                    'blockNumber' : block['NUMBER'], 
                    'from' : transaction['FROM_ADDRESS'], 
                    'quantity' : quantity, 
                    'quoteQuantity' : quote_quantity, 
                    'price' : price, 
                    'marketFeePecentage' : market_fee_pectentage, # the fee can change with CommitNewFee and ApplyNewFee events - need to integrate tracking for that change?
                    'marketFeeValue': market_fee_value,
                    'receivedMS' : block['RECEIVED_TIMESTAMP'], 
                    'fullBlockOnDataApi.ASSET_ID' : block['ASSET_ID'], 
                    'fullBlockOnDataApi.PROVIDER_KEY' : block['PROVIDER_KEY']}

        return internal_trade_object
    
    else: raise KeyError(f"This Pool Address Doesn't Exist in Markets")

In [13]:
def parse_liquidity_update_external(log, events, event_name, instrument_ID, markets_redis):
            
    if event_name == 'TokenExchange':
        swap = decode_raw_log(log, events, 'TokenExchange')

        for pool in markets_redis:
            if pool['instrumentId'] == swap['address']:
                market = pool
                break

        change_in_list = [0] * len(market['tokens'])
        change_in_list[swap['sold_id']] = swap['tokens_sold']
        change_in_list[swap['bought_id']] = -swap['tokens_bought']

        change_in = change_in_list
        invariant = ''
        token_supply = ''
    
    elif event_name == 'TokenExchangeUnderlying':
        swap = decode_raw_log(log, events, 'TokenExchangeUnderlying')

        for pool in markets_redis:
            if pool['instrumentId'] == swap['address']:
                market = pool
                break
            
        change_in_list = [0] * len(market['tokens'])
        change_in_list[swap['sold_id']] = swap['tokens_sold']
        change_in_list[swap['bought_id']] = -swap['tokens_bought']

        change_in = change_in_list
        invariant = ''
        token_supply = ''
    
    elif event_name == 'AddLiquidity':

        mint = decode_raw_log(log, events, 'AddLiquidity')
        length = len(mint['token_amounts'])
        change_in = [mint['token_amounts'][i] for i in range(length)]
        invariant = mint['invariant']
        token_supply = mint['token_supply']
        
    elif event_name == 'RemoveLiquidity':
        burn = decode_raw_log(log, events, 'RemoveLiquidity')
        length = len(burn['token_amounts'])
        change_in = [-burn['token_amounts'][i] for i in range(length)] 
        invariant = ''
        token_supply = burn['token_supply']
    
    elif event_name == 'RemoveLiquidityOne':
        burn_one = decode_raw_log(log, events, 'RemoveLiquidityOne')

        for log_ in transaction['LOGS']:
            if log_['TOPICS'][0] == META_events1['Transfer']['event_hash']: 
                transfer = decode_raw_log(log_, META_events1, 'Transfer')
                if transfer['value'] == burn_one['coin_amount']: 
                    removed_token = transfer['address'].lower()        

        for pool in markets_redis: 
            if pool['instrumentId'] == instrument_ID: 
                if pool['poolType'] == 'MetaPool':
                    if removed_token in pool['tokens']:
                        final_index = 0
                    else: 
                        final_index = 1
                    pool_lengh  = 1
                else:
                    print(pool['tokens'])
                    for index, token in enumerate(pool['tokens']):
                        print(index, token)
                        if token.lower() == removed_token.lower():
                            final_index = index
                    pool_lengh = len(pool['tokens'])

        change_in = [0 for i in range(pool_lengh+1)]
        change_in[final_index] = -burn_one['coin_amount']
        invariant = ''
        
        try: #Registery doesn't have token_supply
            token_supply = burn_one['token_supply']
        except: 
            token_supply = ''

    elif event_name == 'RemoveLiquidityImbalance':
        burn_imbalance = decode_raw_log(log, events, 'RemoveLiquidityImbalance')
        length = len(burn_imbalance['token_amounts'])
        change_in = [-burn_imbalance['token_amounts'][i] for i in range(length)] 
        invariant = burn_imbalance['invariant']
        token_supply = burn_imbalance['token_supply']

    else: 
        raise KeyError(f"event_name: {event_name}, invalid must be ['TokenExchange', 'TokenExchangeUnderlying', 'AddLiquidity', 'RemoveLiquidity', 'RemoveLiquidityOne', 'RemoveLiquidityImbalance']")

    
    parsed_liquidity_update = {
                        'changeIn' : change_in, # a list of changes
                        'invariant' : invariant,
                        'token_supply' : token_supply}
    
    return parsed_liquidity_update
    

def convert_external_liquidity_update_to_internal(events, event_name, block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0):
    
    parsed_external_format = parse_liquidity_update_external(log, events, event_name, instrument_ID, markets_redis)

    change_in = parsed_external_format['changeIn']
    invariant = parsed_external_format['invariant']
    token_supply = parsed_external_format['token_supply']

    executedTS = block['TIMESTAMP']
    executedNS = 0 #.getNSFromTimestamp(executedTS, 'MILLI')

    from_address = transaction['FROM_ADDRESS']
    blockNumber = block['NUMBER']
    transactionHash = transaction['HASH']
    trade_id = f"{transaction['HASH']}-{log['INDEX']}"

    internal_liquidity_update_object = {'exchangeInternalName' : exchange_internal_name, 
                                        'fullBlockOnDataApi.ASSET_ID' : block['ASSET_ID'], 
                                        'instrumentID' : instrument_ID, 
                                        'event' : event_name, 
                                        'tradeID' : trade_id,
                                        'executedTS' : executedTS, 
                                        'executedNS' : executedNS,
                                        'transactionHash' : transactionHash, 
                                        'blockNumber' : blockNumber, 
                                        'from' : from_address, 
                                        'changeIn' : change_in, 
                                        'invariant' : invariant, 
                                        'token_supply' : token_supply, 
                                        'fullBlockOnDataApi.PROVIDER_KEY' : block['PROVIDER_KEY'], 
                                        'receivedMS' : receivedMS,
                                        }

    return internal_liquidity_update_object

In [17]:
trades = []
liquidity_updates = []


for block_number in tqdm([20716772]):
    block = get_CC_block(block_number)
    for transaction in block['TRANSACTIONS']:
        for log in transaction['LOGS']:
            if log['ADDRESS'] in list(pd.DataFrame(markets_redis)['instrumentId']): 

                instrument_ID = log['ADDRESS']
                pool = [pool for pool in markets_redis if pool['instrumentId'] == instrument_ID][0]

                if pool['exchangeInternalName'] == "Curve_Registry": 
                    exchange_internal_name = 'Curve_Registry'
                    events = REG_events11 # In the python integration this is incosnsistent - In the JS integration this is fixed
                    events['TokenExchangeUnderlying'] = {"event_hash": None}
                    events['ApplyNewFee'] = {"event_hash": None}

                elif pool['exchangeInternalName'] == "Curve_MetaPoolFactory":
                    exchange_internal_name = 'Curve_MetaPoolFactory'
                    events = META_events1
                    events['NewFee'] = {"event_hash": None}
                    events['ApplyNewFee'] = {"event_hash": None}

                elif pool['exchangeInternalName'] == 'Curve_StableSwap-NG':
                    exchange_internal_name = 'Curve_StableSwap-NG'
                    events = STABLESWAP_events1
                    events['NewFee'] = {"event_hash": None}
                
                elif pool['exchangeInternalName'] == 'Curve_crvUSD_Factory':
                    exchange_internal_name = 'Curve_crvUSD_Factory'
                    events = CRVUSD_events1
                    events['TokenExchangeUnderlying'] = {"event_hash": None}
                    events['NewFee'] = {"event_hash": None}
                
                else:
                    print(transaction['HASH'])
                    raise KeyError(f"Pools Aren't Part of Factories")
                    
               
                if pool['poolType'] == "MetaPool": 
                    if log['TOPICS'][0] == events['TokenExchange']['event_hash']:
                        swap = convert_external_swap_to_internal(log, events, 'TokenExchange', transaction, block, markets_redis, exchange_internal_name)
                        for log_ in  transaction['LOGS']: 
                            if log_['ADDRESS'] == pool['base_pool_if_meta']: 
                                    
                                    events = REG_events11 
                                    
                                    if log_['TOPICS'][0] == events['AddLiquidity']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'AddLiquidity', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                                    if log_['TOPICS'][0] == events['RemoveLiquidity']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidity', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                                    if log_['TOPICS'][0] == events['RemoveLiquidityOne']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidityOne', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                                    if log_['TOPICS'][0] == events['RemoveLiquidityImbalance']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidityImbalance', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                        swap['quoteQuantity'] = int([i for i in liquidity_update['changeIn'] if i > 0][0])
                        swap['price'] = swap['quoteQuantity']/ swap['quantity']
                        events = META_events1
                        trades.append(swap)

                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'TokenExchange', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                        liquidity_updates.append(liquidity_update) 

                    if log['TOPICS'][0] == events['TokenExchangeUnderlying']['event_hash']:
                        swap = convert_external_swap_to_internal(log, events, 'TokenExchangeUnderlying', transaction, block, markets_redis, exchange_internal_name)
                        
                        for log_ in  transaction['LOGS']: 
                            if log_['ADDRESS'] == pool['base_pool_if_meta']:
                                    
                                    # CLOSEST TO TOKENEXCHANGEUNDERLYING AND THEN ONCE FOUND BREAK 
                                    
                                    events = REG_events11 
                                    
                                    if log_['TOPICS'][0] == events['AddLiquidity']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'AddLiquidity', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                                    if log_['TOPICS'][0] == events['RemoveLiquidity']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidity', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                                    if log_['TOPICS'][0] == events['RemoveLiquidityOne']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidityOne', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                                    if log_['TOPICS'][0] == events['RemoveLiquidityImbalance']['event_hash']: 
                                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidityImbalance', block, transaction, log_, exchange_internal_name, instrument_ID, receivedMS = 0)

                        swap['quoteQuantity'] = int([ i for i in liquidity_update['changeIn'] if i > 0][0])
                        swap['price'] = swap['quoteQuantity']/ swap['quantity']
                        events = META_events1
                        trades.append(swap)

                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'TokenExchangeUnderlying', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                        liquidity_updates.append(liquidity_update)

                else: 
                    if log['TOPICS'][0] == events['TokenExchange']['event_hash']:
                        swap = convert_external_swap_to_internal(log, events, 'TokenExchange', transaction, block, markets_redis, exchange_internal_name)
                        trades.append(swap)

                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'TokenExchange', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                        liquidity_updates.append(liquidity_update) 

                    if log['TOPICS'][0] == events['TokenExchangeUnderlying']['event_hash']:
                        swap = convert_external_swap_to_internal(log, events, 'TokenExchangeUnderlying', transaction, block, markets_redis, exchange_internal_name)
                        trades.append(swap)

                        liquidity_update = convert_external_liquidity_update_to_internal(events, 'TokenExchangeUnderlying', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                        liquidity_updates.append(liquidity_update) 
                
                if log['TOPICS'][0] == events['AddLiquidity']['event_hash']: 
                    liquidity_update = convert_external_liquidity_update_to_internal(events, 'AddLiquidity', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                    liquidity_updates.append(liquidity_update) 

                if log['TOPICS'][0] == events['RemoveLiquidity']['event_hash']: 
                    liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidity', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                    liquidity_updates.append(liquidity_update) 

                if log['TOPICS'][0] == events['RemoveLiquidityOne']['event_hash']: 
                    liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidityOne', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                    liquidity_updates.append(liquidity_update) 

                if log['TOPICS'][0] == events['RemoveLiquidityImbalance']['event_hash']: 
                    liquidity_update = convert_external_liquidity_update_to_internal(events, 'RemoveLiquidityImbalance', block, transaction, log, exchange_internal_name, instrument_ID, receivedMS = 0)
                    liquidity_updates.append(liquidity_update) 

                if log['TOPICS'][0] == events['ApplyNewFee']['event_hash']: 
                    for pool in markets_redis:
                        if pool['instrumentId'] == instrument_ID:
                            pool['fee'] = decode_raw_log(log, events, 'ApplyNewFee')['fee']

                if log['TOPICS'][0] == events['NewFee']['event_hash']: 
                    for pool in markets_redis:
                        if pool['instrumentId'] == instrument_ID:
                            pool['fee'] = decode_raw_log(log, events, 'ApplyNewFee')['fee']

100%|██████████| 1/1 [00:00<00:00,  1.21it/s]


In [18]:
liquidity_updates

[{'exchangeInternalName': 'Curve_Registry',
  'fullBlockOnDataApi.ASSET_ID': 2,
  'instrumentID': '0xdcef968d416a41cdac0ed8702fac8128a64241a2',
  'event': 'AddLiquidity',
  'tradeID': '0xaf2a9696de2d238360452631dfb58428551b75e0e7837f6ebcb6c8bc2522809b-158',
  'executedTS': 1725929735,
  'executedNS': 0,
  'transactionHash': '0xaf2a9696de2d238360452631dfb58428551b75e0e7837f6ebcb6c8bc2522809b',
  'blockNumber': 20716772,
  'from': '0xedefce1a0ea926657376ebc47b1b8c3a60305823',
  'changeIn': [0, 14996011499],
  'invariant': 13211985442270885840639026,
  'token_supply': 13148263885413886989441850,
  'fullBlockOnDataApi.PROVIDER_KEY': 'quicknode',
  'receivedMS': 0},
 {'exchangeInternalName': 'Curve_MetaPoolFactory',
  'fullBlockOnDataApi.ASSET_ID': 2,
  'instrumentID': '0xe57180685e3348589e9521aa53af0bcd497e884d',
  'event': 'TokenExchangeUnderlying',
  'tradeID': '0xaf2a9696de2d238360452631dfb58428551b75e0e7837f6ebcb6c8bc2522809b-160',
  'executedTS': 1725929735,
  'executedNS': 0,
  'tran

In [19]:
trades

[{'exchangeInternalName': 'Curve_MetaPoolFactory',
  'fullBlockOnDataApi.ASSET_ID': 2,
  'instrumentID': '0xe57180685e3348589e9521aa53af0bcd497e884d_2_0x865377367054516e17014ccded1e7d814edc9ce4_0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  'side': 'BUY',
  'tradeID': '0xaf2a9696de2d238360452631dfb58428551b75e0e7837f6ebcb6c8bc2522809b-160',
  'executedTS': 1725929735,
  'executedNS': 0,
  'transactionHash': '0xaf2a9696de2d238360452631dfb58428551b75e0e7837f6ebcb6c8bc2522809b',
  'blockNumber': 20716772,
  'from': '0xedefce1a0ea926657376ebc47b1b8c3a60305823',
  'quantity': 15033981249983403441595,
  'quoteQuantity': 14996011499,
  'price': 9.97474404793245e-13,
  'marketFeePecentage': 0.04,
  'marketFeeValue': '601359249999336177664',
  'receivedMS': 1725929749,
  'fullBlockOnDataApi.PROVIDER_KEY': 'quicknode'}]

In [ ]:
for each log:
    get log address:

        if metqpool:
            exchange_underlying 
            exchange_token_underlying 

        else: 
            exchange_underlying 
            exchange_token_underlying 
    
        add liquidity_update
        remove liquidity 
        fess